In [106]:
from langchain.llms import OpenAI
import os
from dotenv import load_dotenv
# Carregando as variáveis de ambiente do arquivo .env
load_dotenv()

from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

In [107]:
template = """
You are an AI assistant designed to help with questions from the UVA competitive programming book. Your tasks include solving coding problems, providing explanations for solutions, and offering hints to students. Here are the guidelines for your responses:

1. **Understanding the Problem**: Read the problem statement carefully and understand the requirements.
2. **Providing a Solution**: If the problem requires coding, provide a well-commented solution in the requested programming language (usually C++ or Python).
3. **Explaining the Solution**: Explain your approach and the logic behind the solution step-by-step.
4. **Offering Hints**: If the student requests hints instead of a full solution, provide incremental hints that guide them toward the solution without giving it away entirely.
5. **Considering Constraints**: Pay attention to any constraints mentioned in the problem statement, such as time limits, memory limits, and input size.

### Example Questions:

Question: Can you give me some examples for graph questions ?
Answer: Sure! Here are a few example questions from the UVA competitive programming book that involve graph algorithms:

* UVA 558 - Wormholes:
Problem Statement: Given a directed graph representing a series of wormholes, determine if there is a time loop (negative weight cycle).
Solution Approach: This problem can be solved using the Bellman-Ford algorithm, which is capable of detecting negative weight cycles in a graph.
Hint: Implement the Bellman-Ford algorithm and check if you can relax any edge after V-1 iterations, where V is the number of vertices.

* UVA 11902 - Dominator:
Problem Statement: Given a directed graph, determine which vertices are reachable from a given start vertex and find out which vertices can still be reached when each vertex is removed one by one.
Solution Approach: This problem involves performing multiple DFS/BFS searches from the start vertex and simulating the removal of each vertex.
Hint: Start by performing a DFS/BFS from the given start vertex to identify initially reachable vertices. Then, for each vertex, temporarily remove it and perform another DFS/BFS to see the impact.

* UVA 11957 - Checkers:
Problem Statement: Given an N x N board with checkers, find the number of ways a checker can move from the top-left corner to the bottom-right corner.
Solution Approach: Use dynamic programming to count the number of ways to reach each cell, considering the allowed moves of the checker.
Hint: Create a DP table where each cell (i, j) represents the number of ways to reach that cell. Use the moves of the checker to update the table.

Question: Can you help me solve UVA 821 - Page Hopping?
Answer: Absolutely! Let's break down the problem and solve it step-by-step.

* Problem Statement: Given a list of directed edges representing links between pages, calculate the average shortest path length between all pairs of pages. If a page is not reachable from another, it should not be considered in the average.

* Step-by-Step Solution:

1. Understanding the Problem: You need to calculate the shortest paths between all pairs of pages using the given directed edges.
This is a classic application of the Floyd-Warshall algorithm, which is used to find shortest paths in a weighted graph with both positive and negative weights (but no negative weight cycles).

2. Solution in Python:
INF = float('inf')

def floyd_warshall(n, graph):
    dist = [[INF] * n for _ in range(n)]
    for u in range(n):
        dist[u][u] = 0
    for u, v, w in graph:
        dist[u][v] = w
    for k in range(n):
        for i in range(n):
            for j in range(n):
                if dist[i][j] > dist[i][k] + dist[k][j]:
                    dist[i][j] = dist[i][k] + dist[k][j]
    return dist

def average_shortest_path_length(n, dist):
    total_sum = 0
    count = 0
    for i in range(n):
        for j in range(n):
            if i != j and dist[i][j] != INF:
                total_sum += dist[i][j]
                count += 1
    return total_sum / count if count != 0 else 0

# Example usage
n = 4  # number of nodes
graph = [(0, 1, 1), (1, 2, 1), (2, 3, 1), (3, 0, 1)]
dist = floyd_warshall(n, graph)
avg_length = average_shortest_path_length(n, dist)
print('Average shortest path length:', avg_length)

3. Explanation:
Floyd-Warshall Algorithm: Initialize the distance matrix with infinite values, set the distance to zero for self-loops, and update the distance for direct edges. Use three nested loops to update the shortest paths considering each node as an intermediate point.
Average Calculation: Sum the shortest path lengths for all pairs of nodes (excluding self-loops) and divide by the number of valid paths to get the average length.

4. Hints:
Hint 1: Start by initializing the distance matrix and filling in the direct edges.
Hint 2: Use the Floyd-Warshall algorithm to update the shortest paths between all pairs of nodes.
Hint 3: Calculate the average of the shortest path lengths, ignoring any pairs where a path does not exist.

Question: {question}
"""

prompt = PromptTemplate.from_template(template)

In [108]:
llm = OpenAI()

In [109]:
llm = OpenAI(openai_api_key=os.environ.get("OPENAI_API_KEY"))

In [110]:
llm_chain = prompt | llm

In [111]:
llm_chain

PromptTemplate(input_variables=['question'], template="\nYou are an AI assistant designed to help with questions from the UVA competitive programming book. Your tasks include solving coding problems, providing explanations for solutions, and offering hints to students. Here are the guidelines for your responses:\n\n1. **Understanding the Problem**: Read the problem statement carefully and understand the requirements.\n2. **Providing a Solution**: If the problem requires coding, provide a well-commented solution in the requested programming language (usually C++ or Python).\n3. **Explaining the Solution**: Explain your approach and the logic behind the solution step-by-step.\n4. **Offering Hints**: If the student requests hints instead of a full solution, provide incremental hints that guide them toward the solution without giving it away entirely.\n5. **Considering Constraints**: Pay attention to any constraints mentioned in the problem statement, such as time limits, memory limits, an

In [112]:
question = "Name a fill algo for graphs"

print(llm_chain.invoke(question))

Answer: One famous fill algorithm for graphs is the Flood Fill Algorithm, also known as the Seed Fill Algorithm. It is used to determine the area connected to a given node in a multi-dimensional array/graph. The algorithm starts at a given node and checks its adjacent nodes, filling in any empty/white cells with a different color. This process continues until all adjacent nodes have been visited. It is commonly used in image processing and computer graphics. 


# Using Pinecone

In [113]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI

loader = PyPDFLoader("../data/CompetitiveProgramming3.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4",
    chunk_size=100,
    chunk_overlap=0,
)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

In [124]:
from langchain_pinecone import PineconeVectorStore

index_name = "cp3"
embeddings = OpenAIEmbeddings()

docsearch = PineconeVectorStore(index_name=index_name, embedding=embeddings)

In [115]:
query = "Give me questions about graphs"
docs = docsearch.similarity_search(query)
print(docs[1])

page_content='graph. It is a classical problem in graph theory and has many real life applications. For\nexample, we can model the city that we live in as a graph. The vertices are the road\njunctions. The edges are the roads. The time taken to traverse a road is the weight of theedge. You are currently in one road junction. What is the shortest possible time to reach\nanother certain road junction?' metadata={'page': 170.0, 'source': '../data/CompetitiveProgramming3.pdf'}


In [118]:
# Set up QA chain
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 2})
qa = ConversationalRetrievalChain.from_llm(llm_chain, retriever)

In [123]:
print(qa.invoke({"question": "I wanted to learn about Ad-hoc problem, can you give me some questions ?", "chat_history": []})['answer'])

Answer: Sure! Here are a few example questions from the UVA competitive programming book that fall under the Ad Hoc problem category:

* UVA 488 - Triangle Wave:
Problem Statement: Given an amplitude and frequency, print a triangle wave pattern of the specified size.
Solution Approach: Use a combination of loops and string formatting to print the desired pattern.
Hint: Break down the problem into smaller sub-problems, such as printing a single line of the triangle wave at a time.

* UVA 10055 - Hashmat the Brave Warrior:
Problem Statement: Given the number of soldiers in Hashmat's army and the enemy's army, determine the difference in the number of soldiers between the two armies.
Solution Approach: Simply subtract the enemy's army from Hashmat's army and take the absolute value.
Hint: Remember to handle the case where the enemy's army may be larger than Hashmat's army.

* UVA 10911 - Forming Quiz Teams:
Problem Statement: Given a list of students and their coordinates on a grid, find 